# **Building Our Chatbot using llama-index + ChatGPT API**
In this notebook we will be building a chatbot from your own documents. This can be your company documents, FAQs product manuals etc. <br>
You will need a ChatGPT API key. Dont worry you can get a free trial from which will be more than enough to train and test this model. <br>
ChatGPT API: https://platform.openai.com/ <br>
llama-index documentation: https://gpt-index.readthedocs.io/en/latest/guides/primer/usage_pattern.html#optional-save-the-index-for-future-use

## Installing required packages

In [ ]:
!pip install llama_index
!pip install langchain
!pip install docx2txt

## Importing Packages 

In [2]:
import os
from langchain import OpenAI
from llama_index import (
    GPTVectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    QuestionAnswerPrompt,
)

# Set Up OpenAI API Key 
You can get your OpenAI API key from the API website i gave earlier. <br>
Replace "YOUR_OPENAI_API_KEY" with your own key

In [3]:
os.environ["OPENAI_API_KEY"] = 'YOUR_OPENAI_API_KEY'

# Load your documents
Load your documents this can be JSON , word or txt format <br>
NOTE: This is link to a **folder** containing your documents. For my case my documents are in directory named data

In [11]:
documents = SimpleDirectoryReader('/content/data').load_data()

# Customizing Large Language Model's
By default, we use OpenAI's text-davinci-003 model. You may choose to use another LLM when constructing an index. <br>
We set the temperature to 0 to ensure that the responses are always the same.

In [12]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003"))


#Set up Global Servic Context
We are using the Service Context class to define the parameters for the chatbot service. This service context will always be used as the default if not specified as a keyword argument in LlamaIndex functions.

In [13]:
# configure service context
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
# build index
index = GPTVectorStoreIndex.from_documents(
    documents, service_context=service_context
)

# Save Index
Save index so that we can reuse it without recreating it. By default, data is stored in-memory. To save it a directory specify the name of the directory it will be stored there

In [14]:
index.storage_context.persist(persist_dir="index")

# Load Index
To re- load the index from disk use **load_index_from_storage() ** method.

In [15]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="index")

# load index
index = load_index_from_storage(storage_context)

# Query Chatbot 
After building the index, you can now query it with a QueryEngine. Note that a “query” is simply an input to an LLM - this means that you can use the index for question-answering.

In [16]:
query_engine = index.as_query_engine()

response = query_engine.query("Explain the English Rule")
print(response)


The English Rule in New York City refers to the period of time when the city was under the control of the English monarchy. This period began in 1664 when the English captured the Dutch colony of New Amsterdam and renamed it New York after the Duke of York (the future King James II and VII). During this period, the English established a strong presence in the city, with the British military and political base of operations in North America. The English also established a system of slavery in the city, with 42% of households enslaving Africans by 1730. The English also established the Stamp Act Congress in 1765, which was a meeting of representatives from the colonies to discuss taxation. The English Rule in New York City ended in 1783 when the British forces evacuated the city at the close of the American Revolutionary War.
